In [2]:
import pandas as pd
import numpy as np

In [6]:
b2c_forecast = pd.read_csv ('/Users/krishnababu/Desktop/b2c_items_sales_forecast.csv')
icd_forecast = pd.read_csv ('/Users/krishnababu/Desktop/icd_items_sales_forecast.csv')
scd_forecast = pd.read_csv ('/Users/krishnababu/Desktop/scd_items_sales_foreecast.csv')
cross_docking = pd.read_csv ('/Users/krishnababu/Desktop/item_cross_docking_shift.csv')
blocked_items =pd.read_csv ('/Users/krishnababu/Desktop/items_blocked_per_cd_item.csv')

In [8]:
print ("\n B2C Forecast Info")
print (b2c_forecast.info())
print ("\n ICD Forecast Info")
print (icd_forecast.info())
print ("\n SCD Forecast Info")
print (scd_forecast.info())
print ("\n Cross Docking Info")
print (cross_docking.info())
print ("\n Blocked_items Info")
print (blocked_items.info())


 B2C Forecast Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10556 entries, 0 to 10555
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   forecast_date          10556 non-null  object
 1   version                10556 non-null  int64 
 2   scenario_id            10556 non-null  int64 
 3   date                   10556 non-null  object
 4   planning_shop_cluster  10556 non-null  object
 5   shipping_wh            10556 non-null  int64 
 6   sales_forecast_items   10556 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 577.4+ KB
None

 ICD Forecast Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42224 entries, 0 to 42223
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   forecast_date          42224 non-null  object
 1   version                42224 non-null  int64 
 2   scenario_id        

In [10]:
print ("\n --- Missing Values ---")
print ("B2C Forecast: \n", b2c_forecast.isnull().sum())
print ("ICD Forecast: \n", icd_forecast.isnull().sum())
print ("SCD Forecast: \n", scd_forecast.isnull().sum())
print ("Cross_Docking: \n", cross_docking.isnull().sum())
print ("Blocked_Items: \n", blocked_items.isnull().sum())


 --- Missing Values ---
B2C Forecast: 
 forecast_date            0
version                  0
scenario_id              0
date                     0
planning_shop_cluster    0
shipping_wh              0
sales_forecast_items     0
dtype: int64
ICD Forecast: 
 forecast_date            0
version                  0
scenario_id              0
date                     0
planning_shop_cluster    0
shipping_wh              0
picking_wh               0
item_cd                  0
dtype: int64
SCD Forecast: 
 forecast_date            0
version                  0
scenario_id              0
date                     0
planning_shop_cluster    0
shipping_wh              0
picking_wh               0
shipment_cd              0
dtype: int64
Cross_Docking: 
 exported_at_picking_warehouse    0
picking_warehouse_id             0
shipping_warehouse_id            0
day                              0
share                            0
dtype: int64
Blocked_Items: 
 date                               0
shipping

In [12]:
print(b2c_forecast['forecast_date'].dtypes)


object


In [238]:
b2c_forecast ['date'] = pd.to_datetime(b2c_forecast['date'])
icd_forecast ['date'] = pd.to_datetime(icd_forecast['date'])
scd_forecast ['date'] = pd.to_datetime(scd_forecast['date'])
blocked_items ['date'] = pd.to_datetime(blocked_items['date'])


In [240]:
print("B2C Forecast Dates:", b2c_forecast['date'].min(), "to", b2c_forecast['date'].max())
print ("ICD Forecast Dates:", icd_forecast['date'].min(), "to", icd_forecast['date'].max())
print ("SCD Forecast Dates:", scd_forecast['date'].min(), "to", scd_forecast['date'].max())
    
      


       

B2C Forecast Dates: 2024-05-27 00:00:00 to 2024-08-25 00:00:00
ICD Forecast Dates: 2024-05-27 00:00:00 to 2024-08-25 00:00:00
SCD Forecast Dates: 2024-05-27 00:00:00 to 2024-08-25 00:00:00


In [246]:
b2c_exported = (
    b2c_forecast[b2c_forecast['shipping_wh'] == 2]
    .groupby(['date', 'planning_shop_cluster'])['sales_forecast_items']
    .sum()
    .reset_index()
    .rename(columns={'date': 'exported_at', 'sales_forecast_items': 'b2c_items'})
)

In [248]:
print(b2c_exported)

     exported_at planning_shop_cluster  b2c_items
0     2024-05-27                    AT          0
1     2024-05-27                    BE          0
2     2024-05-27                    BG        754
3     2024-05-27                    CH          0
4     2024-05-27                    CY         34
...          ...                   ...        ...
2634  2024-08-25                    RO       1610
2635  2024-08-25                    SE          0
2636  2024-08-25                    SI       1212
2637  2024-08-25                    SK       1575
2638  2024-08-25                    ZZ          0

[2639 rows x 3 columns]


In [252]:
scd_exported = (
    scd_forecast[scd_forecast['picking_wh'] == 2]
    .groupby(['date', 'planning_shop_cluster'])['shipment_cd']
    .count()
    .reset_index()
    .rename(columns={'date': 'exported_at', 'shipment_cd': 'scd_items'})
)

In [254]:
print(scd_exported)

     exported_at planning_shop_cluster  scd_items
0     2024-05-27                    AT          4
1     2024-05-27                    BE          4
2     2024-05-27                    BG          4
3     2024-05-27                    CH          4
4     2024-05-27                    CY          4
...          ...                   ...        ...
2634  2024-08-25                    RO          4
2635  2024-08-25                    SE          4
2636  2024-08-25                    SI          4
2637  2024-08-25                    SK          4
2638  2024-08-25                    ZZ          4

[2639 rows x 3 columns]


In [256]:
icd_items = (
    icd_forecast[icd_forecast['shipping_wh'] == 2]
    .groupby(['date', 'planning_shop_cluster'])['item_cd']
    .count()
    .reset_index()
    .rename(columns={'date': 'exported_at', 'item_cd': 'icd_items'})
)

In [258]:
print(icd_items)

     exported_at planning_shop_cluster  icd_items
0     2024-05-27                    AT          4
1     2024-05-27                    BE          4
2     2024-05-27                    BG          4
3     2024-05-27                    CH          4
4     2024-05-27                    CY          4
...          ...                   ...        ...
2634  2024-08-25                    RO          4
2635  2024-08-25                    SE          4
2636  2024-08-25                    SI          4
2637  2024-08-25                    SK          4
2638  2024-08-25                    ZZ          4

[2639 rows x 3 columns]


In [260]:
items_minus = (
    icd_items.merge(blocked_items, left_on ='exported_at', right_on='date', how='left')
    .assign(items_minus=lambda df: df['icd_items'] - df['items_per_cross_docking_cd_item'])
    [['exported_at', 'planning_shop_cluster', 'items_minus']]
)

In [262]:

print(cross_docking['day'].unique())


<DatetimeArray>
['2024-06-01 00:00:00', '2024-06-02 00:00:00', '2024-06-03 00:00:00',
 '2024-06-04 00:00:00', '2024-06-05 00:00:00', '2024-06-06 00:00:00',
 '2024-06-07 00:00:00']
Length: 7, dtype: datetime64[ns]


In [264]:
print(cross_docking.head())

  exported_at_picking_warehouse  picking_warehouse_id  shipping_warehouse_id  \
0                    2024-05-27                     1                      2   
1                    2024-05-27                     1                      2   
2                    2024-05-27                     1                      2   
3                    2024-05-27                     1                      2   
4                    2024-05-27                     1                      2   

         day  share  
0 2024-06-01   0.02  
1 2024-06-02   0.23  
2 2024-06-03   0.61  
3 2024-06-04   0.14  
4 2024-06-05   0.00  


In [266]:
items_plus = (
    items_minus.merge(cross_docking, left_on ='exported_at', right_on = 'day', how = 'left')
    .assign(items_plus=lambda df: df['items_minus'] * df['share'])
    .groupby(['exported_at', 'planning_shop_cluster'])['items_plus']
    .sum()
    .reset_index()
)
            

In [268]:
final_forecast = (
    b2c_exported 
    .merge(scd_exported, on = ['exported_at', 'planning_shop_cluster'], how = 'outer')
    .merge(items_minus, on = ['exported_at', 'planning_shop_cluster'], how = 'outer')
    .merge(items_plus, on = ['exported_at','planning_shop_cluster'], how = 'outer')
    .fillna(0)
)

In [270]:
final_forecast['final_items'] = (
    final_forecast['b2c_items'] +
    final_forecast['scd_items'] -
    final_forecast['items_minus'] +
    final_forecast['items_plus']
)
    

In [272]:
final_forecast_pivot = final_forecast.pivot_table(
    index = 'planning_shop_cluster' ,
    columns = 'exported_at' , 
    values = 'final_items' ,
    aggfunc = 'sum'
)

In [274]:
final_forecast_pivot = final_forecast_pivot.fillna(0)

In [276]:
print (final_forecast_pivot)

exported_at            2024-05-27  2024-05-28  2024-05-29  2024-05-30  \
planning_shop_cluster                                                   
AT                         259.88      259.88      259.88      259.88   
BE                         259.88      259.88      259.88      259.88   
BG                       93755.88    87679.88   105163.88   117439.88   
CH                         259.88      259.88      259.88      259.88   
CY                        4475.88     5343.88     6335.88     6583.88   
CZ                         259.88      259.88      259.88      259.88   
DE                         259.88      259.88      259.88      259.88   
DK                         259.88      259.88      259.88      259.88   
EE                         259.88      259.88      259.88      259.88   
ES                         259.88      259.88      259.88      259.88   
FI                         259.88      259.88      259.88      259.88   
FR                         259.88      259.88      

In [278]:
final_forecast_pivot = final_forecast_pivot.round(0).astype(int)

In [280]:
print(final_forecast_pivot)

exported_at            2024-05-27  2024-05-28  2024-05-29  2024-05-30  \
planning_shop_cluster                                                   
AT                            260         260         260         260   
BE                            260         260         260         260   
BG                          93756       87680      105164      117440   
CH                            260         260         260         260   
CY                           4476        5344        6336        6584   
CZ                            260         260         260         260   
DE                            260         260         260         260   
DK                            260         260         260         260   
EE                            260         260         260         260   
ES                            260         260         260         260   
FI                            260         260         260         260   
FR                            260         260      

In [282]:
final_forecast_pivot.to_csv('final_forecast_by_planning_shop_cluster.csv')

In [284]:
print (final_forecast_pivot.head())

exported_at            2024-05-27  2024-05-28  2024-05-29  2024-05-30  \
planning_shop_cluster                                                   
AT                            260         260         260         260   
BE                            260         260         260         260   
BG                          93756       87680      105164      117440   
CH                            260         260         260         260   
CY                           4476        5344        6336        6584   

exported_at            2024-05-31  2024-06-01  2024-06-02  2024-06-03  \
planning_shop_cluster                                                   
AT                            260      501514     6528279    10188432   
BE                            260      501514     6528279    10188432   
BG                         123764      621050     6643351    10263824   
CH                            260      501514     6528279    10188432   
CY                           7204      506598     